In [1]:
import pandas as pd
from ifrs17_template.template import *

### Best Estimate

BestEstimate: 

    LockedBestEstimate if ValuationApproach == 'BBA' and IsOci == True else CurrentBestEstimate
    

LockedBestEstimate:

    EstimateType == 'BE'
    EconomicBasis ==  'L'
    

CurrentBestEstimate:

    EstimateType == 'BE'
    EconomicBasis ==  'C'

### Risk Adjustment


RiskAdjustment:

    LockedRiskAdjustment if ValuationApproach == 'BBA' and IsOci == True else CurrentRiskAdjustment
    EstimateType == 'RA'
    

LockedRiskAdjustment:

    EstimateType == 'RA'
    EconomicBasis ==  'L'
    

CurrentRiskAdjustment:

    EstimateType == 'RA'
    EconomicBasis ==  'C'


## Fulfillment Cash flows

Fcf:

    BestEstimate + RiskAdjustment

LockedFcf:

    LockedBestEstimate + LockedRiskAdjustment
    
CurrentFcf:

    CurrentBestEstimate + CurrentRiskAdjustment

## LRC Technical Margin: CSM, LC, LoReCo

Csm:

    EstimateType == 'C'
    
Lc: 

    EstimateType == 'L'    
    
Loreco:

    EstimateType == 'LR' 


LrcTechnicalMargin

    Lc + Loreco - 1 * Csm

## Written, Accruals and Deferrals 

Written:

    EstimateType == 'A' 

Advance:

    EstimateType == 'AA' 

Overdue:

    EstimateType == 'OA' 

Deferrals

    EstimateType == 'DA' 

## Experience Adjustment

ExperienceAdjustment:

    Written[AocType == 'CF'] - BestEstimate[AocType == 'CF' & Novelty == 'C']

## LIC Actuarial (Actuarial Liability for Incurred Claims)

LicActuarial:

    CurrentFcf[EstimateType == 'LIC']

## LIC (Liability for Incurred Claims)

Lic:

    bop + delta + eop where
        bop: licData[AocType == 'BOP' & Novelty == 'I']
        delta: licData[AocType != 'BOP' & AocType != 'EOP'] + licData[AocType == 'BOP' & Novelty == 'N']
        eop: licData[AocType == 'EOP']
            where
            licData: licActuarial + accrual
                where
                accrual: Advance[LiabilityType == 'LIC']
        

## LRC Actuarial (Actuarial Liability for Remaining Coverage)

LrcActuarial:

    CurrentFcf[LiabilityType == 'LRC'] + Csm + Loreco

## LRC (Liability for Remaining Coverage)

Lrc:

    bop + delta + eop where
        bop:  lrcData[(AocType == 'BOP') & (Novelty == 'I')]
        delta: lrcData[(AocType != 'BOP') & (AocType != 'EOP')] + lrcData[(AocType == 'BOP') & (Novelty == 'N')]
        eop: lrcData[(AocType == 'BOP') & (Novelty == 'I')] where
            lrcData: LrcActuarial + accrual
            accrual: Advance[LiabilityType == 'LRC'] + Overdue[LiabilityType == 'LRC']

## Fulfillment Cash flows (excluding the Loss Component)

FcfChangeInEstimate:

## Contractual Service Margin (CSM)


CsmChangeInEstimate:

## Loss Component (LC)


LcChangeInEstimate:

## Loss Recovery Component (LoReCo)

LorecoChangeInEstimate

## Incurred Actuals

IncurredActuals

## Incurred Deferrals and Acquisition Expenses

IncurredDeferrals

## Experience Adjustment on Premium (allocation to CSM vs P&L recognition)

ExperienceAdjustmentOnPremium

## Financial Performance

FinancialPerformance:

    FcfChangeInEstimate + CsmChangeInEstimate + LcChangeInEstimate + LorecoChangeInEstimate + IncurredActuals + IncurredDeferrals + ExperienceAdjustmentOnPremium
